In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
import pandas as pd
import numpy as np
import re
import folium as folium

In [48]:
df_raw = pd.read_csv('/content/drive/My Drive/mit-covid-hackathon/data-and-notebooks/raceDataTrackerDeaths.csv', 
                 names = ['Date', 'State', 'Update Time', 'Source', 'Update Time (Et)',
       'Last Check', 'Checker', 'DC', 'Total', 'White ', 'Black ',
       'LatinX_Hispanic', 'Asian', 'AIAN', 'NHPI',
       'Multiracial', 'Other', 'Unknown',
       'Race_Percent_Unknown', 'Ethnicity_Hispanic', 'Ethnicity_Non-Hispanic',
       'Ethnicity_Unknown', 'Ethnicity_Percent_Unknown', 'Notes'], low_memory=False) 
df_raw.columns

Index(['Date', 'State', 'Update Time', 'Source', 'Update Time (Et)',
       'Last Check', 'Checker', 'DC', 'Total', 'White ', 'Black ',
       'LatinX_Hispanic', 'Asian', 'AIAN', 'NHPI', 'Multiracial', 'Other',
       'Unknown', 'Race_Percent_Unknown', 'Ethnicity_Hispanic',
       'Ethnicity_Non-Hispanic', 'Ethnicity_Unknown',
       'Ethnicity_Percent_Unknown', 'Notes'],
      dtype='object')

In [49]:
df_raw.head

<bound method NDFrame.head of          Date  State  ...  Ethnicity_Percent_Unknown  Notes
0        Date  State  ...  Ethnicity_Percent_Unknown  Notes
1    20200510     AK  ...                        100    NaN
2    20200510     AL  ...                         11    NaN
3    20200510     AR  ...                        100    NaN
4    20200510     AS  ...                        100    NaN
..        ...    ...  ...                        ...    ...
500  20200412     VT  ...                        NaN    NaN
501  20200412     WA  ...                         26    NaN
502  20200412     WI  ...                          1    NaN
503  20200412     WV  ...                        NaN    NaN
504  20200412     WY  ...                        NaN    NaN

[505 rows x 24 columns]>

In [50]:
# Drop media columns 
df_raw.drop(['Update Time', 'Source', 'Update Time (Et)',
       'Last Check', 'Checker', 'DC','Notes'], axis=1, inplace=True)
df_raw.columns

Index(['Date', 'State', 'Total', 'White ', 'Black ', 'LatinX_Hispanic',
       'Asian', 'AIAN', 'NHPI', 'Multiracial', 'Other', 'Unknown',
       'Race_Percent_Unknown', 'Ethnicity_Hispanic', 'Ethnicity_Non-Hispanic',
       'Ethnicity_Unknown', 'Ethnicity_Percent_Unknown'],
      dtype='object')

In [51]:
df_raw.describe()

,Date,State,Total,White,Black,LatinX_Hispanic,Asian,AIAN,NHPI,Multiracial,Other,Unknown,Race_Percent_Unknown,Ethnicity_Hispanic,Ethnicity_Non-Hispanic,Ethnicity_Unknown,Ethnicity_Percent_Unknown
count,505,505,311,296,284,132,241,115,85,86,248,289,309,217,219,214,224
unique,10,57,265,231,187,101,77,26,15,26,71,142,49,101,191,136,57
top,20200503,SD,9,36,0,5,2,1,1,0,2,0,0,5,3155,0,0
freq,56,9,4,5,8,7,30,28,24,17,25,20,42,17,3,9,20


In [52]:
# display only the number of rows of the ‘df’ DataFrame
df_raw.shape[0]

505

In [0]:
# df with just most recent date, as info is cumulative
df = df_raw[df_raw.Date == '20200510']

In [54]:
# display only the number of rows of the ‘df’ DataFrame
df.shape[0]

56

In [55]:
# count the number of unique values
df.State.nunique()  

56

56 because this data includes 6 of 9 additional US commonwealth & terrirories: 'AS' =   'DC',  'GU',  'MP', 'PR', 'VI' 
* American Samoa - AS
* District of Columbia - DC
* Federated States of Micronesia - FM
* Guam - GU
* Marshall Islands - MH
* Northern Mariana Islands - MP
* Palau - PW
* Puerto Rico - PR
* Virgin Islands - VI


In [56]:
# return the unique values
df.State.unique()    

array(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
       'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV',
       'WY'], dtype=object)

In [57]:
# lowercase column names by mapping the lowering function to all column names
df.columns = map(str.lower, df.columns)
df.head()

,date,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown,ethnicity_hispanic,ethnicity_non-hispanic,ethnicity_unknown,ethnicity_percent_unknown
1,20200510,AK,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,10,100
2,20200510,AL,393,194,178,NaN,3,NaN,NaN,NaN,5,9,2,9,341,42,11
3,20200510,AR,82,45,30,NaN,NaN,NaN,0,NaN,4,3,4,NaN,NaN,82,100
4,20200510,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,100
5,20200510,AZ,536,240,17,77,5,95,NaN,NaN,7,95,18,77,364,95,18


In [58]:
df.describe()

,date,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown,ethnicity_hispanic,ethnicity_non-hispanic,ethnicity_unknown,ethnicity_percent_unknown
count,56,56,55,41,39,18,33,16,11,10,36,54,55,35,35,38,39
unique,1,56,54,41,38,18,23,12,7,9,25,46,25,31,35,33,25
top,20200510,TN,67,211,1,363,6,1,2,10,2,0,100,5,5868,2,100
freq,56,1,2,1,2,1,6,3,2,2,4,5,14,3,1,2,4


In [59]:
# count number of non NA values for race_percent_unknown
df['race_percent_unknown'].count()
# Missing race data for 1 state ()

55

In [60]:
# count number of non NA values for ethnicity_percent_unknown
df['ethnicity_percent_unknown'].count()
# Missing ethnicity data for 7 states/territories

39

In [61]:
# Drop ethnicity columns as group is focusing analysis on race
df.drop(['date', 'ethnicity_hispanic',	'ethnicity_non-hispanic', 'ethnicity_unknown', 'ethnicity_percent_unknown'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [62]:
df.columns

Index(['state', 'total', 'white ', 'black ', 'latinx_hispanic', 'asian',
       'aian', 'nhpi', 'multiracial', 'other', 'unknown',
       'race_percent_unknown'],
      dtype='object')

In [63]:
df.describe()

,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
count,56,55,41,39,18,33,16,11,10,36,54,55
unique,56,54,41,38,18,23,12,7,9,25,46,25
top,TN,67,211,1,363,6,1,2,10,2,0,100
freq,1,2,1,2,1,6,3,2,2,4,5,14


In [64]:
# For how many states/regions are we missing data for one or more races?
df.isna().sum()

state                    0
total                    1
white                   15
black                   17
latinx_hispanic         38
asian                   23
aian                    40
nhpi                    45
multiracial             46
other                   20
unknown                  2
race_percent_unknown     1
dtype: int64

These races have particularly high amounts of missing data 
*   multiracial             46 (missing data 82% of the time) 
*   nhpi                    45 (missing data 80% of the time)
*   aian                    40 (missing data 71% of the time)
*   latinx_hispanic         38 (missing data 68% of the time)
*   asian                   23 (missing data 41% of the time)
*   black                   17 (missing data 30% of the time)

In [65]:
pd.value_counts(df['race_percent_unknown'].values, sort=True)

100    14
0       5
3       4
5       3
4       3
2       2
1       2
13      2
7       2
10      2
20      2
9       1
8       1
19      1
11      1
46      1
66      1
35      1
12      1
23      1
52      1
56      1
6       1
18      1
94      1
dtype: int64

In [66]:
df.sort_values('race_percent_unknown', ascending=False, inplace=True)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
55,WV,50,2,1,NaN,NaN,NaN,NaN,NaN,NaN,47,94
53,WA,931,622,28,75,79,7,5,10,19,86,9
10,DE,224,132,58,13,1,NaN,NaN,NaN,3,17,8
21,LA,2213,833,1184,NaN,16,0,4,NaN,16,157,7
7,CO,971,619,63,161,37,3,3,10,6,63,7
48,TX,1088,176,66,117,6,NaN,NaN,NaN,1,722,66
23,MD,1538,637,631,109,51,NaN,NaN,NaN,22,88,6
34,NH,133,54,0,4,0,NaN,NaN,NaN,1,74,56
42,PA,3707,1357,388,NaN,40,NaN,NaN,NaN,11,1911,52
31,NC,547,312,177,NaN,6,1,1,NaN,20,30,5


In [67]:
df.race_percent_unknown.values

array(['94', '9', '8', '7', '7', '66', '6', '56', '52', '5', '5', '5',
       '46', '4', '4', '4', '35', '3', '3', '3', '3', '23', '20', '20',
       '2', '2', '19', '18', '13', '13', '12', '11', '100', '100', '100',
       '100', '100', '100', '100', '100', '100', '100', '100', '100',
       '100', '100', '10', '10', '1', '1', '0', '0', '0', '0', '0', nan],
      dtype=object)

In [68]:
# Looking at the data, the state with race_percent_unknown "NaN" is AK, 
# which has 10 deaths and no info under any race, 
# so update the spreadsheet with 100 percent unknown for AK. 
df['race_percent_unknown'].replace(float('nan'), 100, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [69]:
# Verify the update
df.head()

,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
55,WV,50,2,1,NaN,NaN,NaN,NaN,NaN,NaN,47,94
53,WA,931,622,28,75,79,7,5,10,19,86,9
10,DE,224,132,58,13,1,NaN,NaN,NaN,3,17,8
21,LA,2213,833,1184,NaN,16,0,4,NaN,16,157,7
7,CO,971,619,63,161,37,3,3,10,6,63,7


In [70]:
df.dtypes

state                   object
total                   object
white                   object
black                   object
latinx_hispanic         object
asian                   object
aian                    object
nhpi                    object
multiracial             object
other                   object
unknown                 object
race_percent_unknown    object
dtype: object

In [71]:
df.head()

,state,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
55,WV,50,2,1,NaN,NaN,NaN,NaN,NaN,NaN,47,94
53,WA,931,622,28,75,79,7,5,10,19,86,9
10,DE,224,132,58,13,1,NaN,NaN,NaN,3,17,8
21,LA,2213,833,1184,NaN,16,0,4,NaN,16,157,7
7,CO,971,619,63,161,37,3,3,10,6,63,7


In [72]:
df = df.set_index('state')
df.head()

,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
state,,,,,,,,,,,
WV,50,2,1,NaN,NaN,NaN,NaN,NaN,NaN,47,94
WA,931,622,28,75,79,7,5,10,19,86,9
DE,224,132,58,13,1,NaN,NaN,NaN,3,17,8
LA,2213,833,1184,NaN,16,0,4,NaN,16,157,7
CO,971,619,63,161,37,3,3,10,6,63,7


In [73]:
# For visualization, drop rows that are not contiguous 50 states
df.drop(['AS', 'DC', 'GU', 'MP', 'PR', 'VI'], inplace=True)
df.shape

(50, 11)

In [0]:
df = df.apply(pd.to_numeric)

# df = df.astype({'state': str, 'total': int, 'white ': int, 'black ': int, 'latinx_hispanic': int, 'asian': int,
#        'aian': int, 'nhpi': int, 'multiracial': int, 'other': int, 'unknown': int,
#        'race_percent_unknown': int}) 

In [75]:
df.dtypes

total                     int64
white                   float64
black                   float64
latinx_hispanic         float64
asian                   float64
aian                    float64
nhpi                    float64
multiracial             float64
other                   float64
unknown                 float64
race_percent_unknown      int64
dtype: object

14 states have 100% unknown race.

In [76]:
# Sort by most unknown
df.sort_values('race_percent_unknown', ascending=False)

,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
state,,,,,,,,,,,
AK,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100
SD,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,100
HI,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,100
NM,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,100
ME,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,100
ND,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,100
NE,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,100
NV,306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306.0,100
WY,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,100


In [77]:
# Show when unknown race > 30 %
df[df['race_percent_unknown'] > 30]

,total,white,black,latinx_hispanic,asian,aian,nhpi,multiracial,other,unknown,race_percent_unknown
state,,,,,,,,,,,
WV,50,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,47.0,94
TX,1088,176.0,66.0,117.0,6.0,NaN,NaN,NaN,1.0,722.0,66
NH,133,54.0,0.0,4.0,0.0,NaN,NaN,NaN,1.0,74.0,56
PA,3707,1357.0,388.0,NaN,40.0,NaN,NaN,NaN,11.0,1911.0,52
MA,4979,1911.0,253.0,293.0,84.0,NaN,NaN,NaN,154.0,2280.0,46
NY,21478,5423.0,3409.0,3738.0,845.0,NaN,NaN,NaN,575.0,7485.0,35
SD,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,100
UT,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,100
WY,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,100


In [0]:
# Drop columns not using for visualization
df.drop(['total', 'white ', 'black ', 'latinx_hispanic', 'asian', 'aian', 'nhpi',
       'multiracial', 'other', 'unknown'],axis=1, inplace=True)


In [0]:
# Write df to csv in gdrive via colab for map
with open('/content/drive/My Drive/Colab Notebooks/df.csv', 'w') as f:
  df.to_csv(f)

21 territories have major gaps (> 30% unknown) for the racial data reported.

In [80]:
# JSON overlay for choropleth map
base = 'https://github.com/python-visualization/folium/raw/master/examples/data/'
antarctic_ice_edge = base + 'antarctic_ice_edge.json'

m = folium.Map(
    location=[-59.1759, -11.6016],
    tiles='Mapbox Bright',
    zoom_start=2
)

folium.GeoJson(
    antarctic_ice_edge,
    name='geojson'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [81]:
state_geo = base + 'us-states.json'

#state_race_gaps = base + 'df_fewm2.csv' # path is '/content/drive/My Drive/Colab Notebooks/df_fewm2.csv'
state_race_gaps = '/content/drive/My Drive/Colab Notebooks/df.csv'
state_data = pd.read_csv(state_race_gaps)

m = folium.Map(location=[48, -102], zoom_start=3)

m.choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['state', 'race_percent_unknown'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Race Unknown Rate (%)'
)

folium.LayerControl().add_to(m)

m

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
